## MLflow Integration for Model Serving and Registry Management



### Objective:
* Loading and Serving Models
* Inference with the Model
* Deleting Models and Versions

### Environment Setup

Ensure all necessary libraries are installed and imported for our workflow.

In [1]:
!pip install mlflow==2.7.1
!pip install transformers==4.43.3

  Using cached transformers-4.43.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.43.3-py3-none-any.whl (9.4 MB)
Using cached tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.3
    Uninstalling transformers-4.52.3:
      Successfully uninstalled transformers-4.52.3


In [2]:
!pip install boto3 awscli

In [ ]:
!aws configure

In [4]:
!pip show transformers

Name: transformers
Version: 4.43.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, sentence-transformers


### Imports

Import necessary libraries focusing on MLflow for model retrieval, PyTorch for model operations, and Transformers for data processing.

In [3]:
# !pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 106.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 108.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.43.3
    Uninstalling transformers-4.43.3:
      Successfully uninstalled transformers-4.43.3


In [5]:
import mlflow
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import os

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:373: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

### Connect to Mlflow Server

In [6]:
# Set MLflow tracking URI
mlflow.set_tracking_uri("http://ec2-174-129-55-125.compute-1.amazonaws.com:5000/")
client = mlflow.tracking.MlflowClient()

### Retrieve the Model from MLflow

In this step, we'll explore two methods to retrieve our trained model from MLflow. Understanding the nuances of each method is key to making an informed choice in a real-life scenario based on the requirements and constraints of your deployment environment.

#### Method 1: Using the Built-in PyTorch Loader

This method is straightforward and uses MLflow's built-in functionality to load PyTorch models. It's user-friendly and works well when you're working within a PyTorch-centric workflow.


In [7]:
# Load a specific model version
model_name = "agnews-transformer"
model_version = "1"  # or "production", "staging"


model_uri = f"models:/{model_name}/{model_version}"
model = mlflow.pytorch.load_model(model_uri)

2025/05/29 11:27:09 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


## Performing Inference

Here, we define the `predict` function to perform inference using the loaded model. This function takes a list of texts, tokenizes them using a pre-trained tokenizer, and then feeds them into the model. The output is the model's prediction, which can be used for various applications such as text classification, sentiment analysis, etc. This step is crucial in demonstrating how a trained model can be utilized for practical applications.


In [8]:

def predict(texts, model, tokenizer):
    # Tokenize the texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(model.device)

    # Pass the inputs to the model
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)

    # Convert predictions to text labels
    predictions = predictions.cpu().numpy()
    predictions = [model.config.id2label[prediction] for prediction in predictions]

    # Print predictions
    return predictions


In [9]:
# Sample text to predict
texts = [
    "The local high school soccer team triumphed in the state championship, securing victory with a last-second winning goal.",
    "DataCore is set to acquire startup InnovateAI for $2 billion, aiming to enhance its position in the artificial intelligence market.",
]


In [10]:
# Tokenizer needs to be loaded sepparetly for this
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

print(predict(texts, model, tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


['World', 'World']


## Cleaning Up: Deleting Models and Versions

In some scenarios, you might need to delete specific model versions or even entire registered models from MLflow. This section covers how to perform these deletions. Note that this should be done cautiously, as it cannot be undone. This is particularly useful for maintaining a clean and efficient model registry by removing outdated or unused models and versions.


In [ ]:
# Delete a specific model version
client.delete_model_version(name=model_name, version=model_version)

# Delete the entire registered model
client.delete_registered_model(name=model_name)
